## Парсер новостей с VC.RU на PYTHON

In [1]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеку для парсинга сайтов !!!!!!!!!!!!!
import requests

In [9]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеки для парсинга сайтов !!!!!!!!!!!!!
import requests
import lxml
import os
import json
import csv

# Работа с файлами XLSX
import openpyxl
import xlsxwriter

# Использование объекта datetime :
from datetime import datetime
from time import sleep
import time
import random

In [2]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/popular'

In [ ]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/new'

# ФУНКЦИЯ № 1. Определение переменных

In [5]:
def variable():
# 0. Создаём словарь заголовков
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    
# 1. Список адресов основного хоста
    hosts = ['https://vc.ru']
    
# 2. Список адресов страниц для парсинга
    urls = ['https://vc.ru/new']

# 3. Список имен сайтов - список словарей
    names = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
    
# 4. Словарь путей для папок с данными
    type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xlsx']
    path_files = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xlsx/'
                ]
    extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xlsx']
    
    paths = [{'type_file': type_files[i], 'path_file': path_files[i], 'extension_file': extension_files[i]} for i in range(len(type_files))]
    
# 5. Словарь атрибутов Супа - список списков
    divs = ['div', 'p']
    classes = ['desc list', 'news-item']
    pattern = [[divs[i], classes[i]] for i in range(len(divs))]
    
    
    return headers, hosts, urls, names, paths, pattern

In [7]:
print(variable()[3][0]['name'])
# print(variable()[3][0])

vc


# ФУНКЦИЯ № 2. Сохранение HTML-кода для тестирования

In [13]:
# import os
# Отправляем запрос на страницу, передаём в качестве GET-параметра URL-адрес
# Сохраним нашу страницу в файл И ЗАКОММЕНТИРУЕМ ЭТУ КОМАНДУ за ненадобностью

def write_html(headers, url, path):
    # path = '../data/interim/html/'
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path[1]['path_file']):
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" does NOT exist - it will be created !!!')
        os.mkdir(path[1]['path_file'])
    else:
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" FOUND!!! It`s OK')
    
    print(f'Fun.2.0. START of function "WRITE_HTML()"\n')
    
    count = 1
# Проходим циклом по списку со ссылками на страницы сайтов
    for u in url:
        r = requests.get(url=u, headers=headers)
        src = r.text

        file_start = 'news_articles_'
        name = u.split('//')[1].split('.')[0]
        file_name = file_start + name
        file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']
        print(f'{count}. Function "WRITE_HTML({name})" EXECUTING ...')

# Проверка существования старых файлов с таким же названием (если они существуют - обновляем)
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[1]['path_file']}\" - It will be CREATED.")
        else:
            print(f"\tIn the folder \"{path[1]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
            os.remove(file_path)
            
# Сохраняем код по указанному пути
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(src)
        print(f'\tHTML-code \"{file_name}\" is SAVED in folder \"{path[1]['path_file']}\"')
        print(f'Function "WRITE_HTML({name.upper()})" is EXECUTED\n')
        count += 1

    print(f'Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!')

# Безусловный запуск функции
if __name__ == '__main__':
    write_html(variable()[0], variable()[2], variable()[4])

Fun.2.0. Folder "../data/interim/html/" FOUND!!! It`s OK
Fun.2.0. START of function "WRITE_HTML()"

1. Function "WRITE_HTML(vc)" EXECUTING ...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	The file "news_articles_vc" will be DELETED and REWRITTEN.
	HTML-code "news_articles_vc" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(VC)" is EXECUTED

Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!


# ФУНКЦИЯ № 3.0. Чтение HTML-кода, сохранение его в список

In [10]:
# READ_HTML:
def read_html(name, path):
    print(f'Fun.3.0. START of function "READ_HTML()"')
    count = 1
    
    # print(name)
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'{count}. Function "READ_HTML({name.upper()})" is EXECUTING...')
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['path_file']}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['path_file']}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    count += 1
    print(f'Fun.3.0. "READ_HTML({name.upper()})" COMPLETED SUCCESSFULLY!\n')

    return src

# Безусловный запуск функции
if __name__ == '__main__':
    read_html(variable()[3][0]['name'], variable()[4][1])

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(VC)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	HTML-code "news_articles_vc" is OPENED from ../data/interim/html/news_articles_vc.html
Fun.3.0. "READ_HTML(VC)" COMPLETED SUCCESSFULLY!



In [3]:
# Создаём запрос
# response = requests.get(url)

In [6]:
# Объект SOUP
# soup = BeautifulSoup(response.text, 'html')
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')

# Вылезает куча кода в виде текста
# print(soup)

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(VC)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	HTML-code "news_articles_vc" is OPENED from ../data/interim/html/news_articles_vc.html
Fun.3.0. "READ_HTML(VC)" COMPLETED SUCCESSFULLY!


In [8]:
# Определяем DIV-ы с нужной инфой (заголовки статей)
# articles = soup.find('div', class_ = 'content content--short')
articles = soup.find('div', class_ = 'content-list')
print (articles)

<div class="content-list"><!--[--><!--[--><!-- --><!-- --><div class="content content--short"><!-- --><!-- --><div class="content-header"><div class="author author--with-sticker" style="--90f1d586:40px;"><a class="author__avatar" data-router-link="" href="/u/597714-polina-laakso"><!--[--><div class="andropov-media andropov-media--rounded andropov-media--bordered andropov-media--has-preview andropov-video" data-loaded="false" style="aspect-ratio:1;width:40px;height:40px;max-width:none;--background-color:#877458;"><!-- --><!--[--><video autoplay="" loop="" muted="" playsinline="true" preload="auto"></video><!--]--></div><!--]--></a><a class="author__avatar-sticker" data-router-link="" href="/offline"><!--[--><div class="andropov-media andropov-media--rounded andropov-media--bordered andropov-media--has-preview andropov-image" data-loaded="true" style="aspect-ratio:1;width:20px;height:20px;max-width:none;--background-color:#f0cab7;"><!-- --><!--[--><picture><source srcset="https://leonard

In [43]:
# Чтобы определить степени вложенности DIV-ов применяем трюк PRETTIFY:
# print(articles.prettify())

In [9]:
# Достаём нужный DIV:
# div = articles.div.div.div
div = articles.find('div', class_ = 'content--short')
print (div)

<div class="content content--short"><!-- --><!-- --><div class="content-header"><div class="author author--with-sticker" style="--90f1d586:40px;"><a class="author__avatar" data-router-link="" href="/u/597714-polina-laakso"><!--[--><div class="andropov-media andropov-media--rounded andropov-media--bordered andropov-media--has-preview andropov-video" data-loaded="false" style="aspect-ratio:1;width:40px;height:40px;max-width:none;--background-color:#877458;"><!-- --><!--[--><video autoplay="" loop="" muted="" playsinline="true" preload="auto"></video><!--]--></div><!--]--></a><a class="author__avatar-sticker" data-router-link="" href="/offline"><!--[--><div class="andropov-media andropov-media--rounded andropov-media--bordered andropov-media--has-preview andropov-image" data-loaded="true" style="aspect-ratio:1;width:20px;height:20px;max-width:none;--background-color:#f0cab7;"><!-- --><!--[--><picture><source srcset="https://leonardo.osnova.io/808531a2-36b0-523a-ab43-9b889f8d3d6c/-/scale_c

In [47]:
# author__name
author = div.find('a', class_='author__name').text
print(author)

Полина Лааксо


In [11]:
# Достаём текст заголовка из этого DIV-а
# title = div.text
title = div.find('div', class_='content-title content-title--low-indent').text
print(title)

Умер «отец Nutella» — химик Франческо Ривелла 


In [39]:
# Краткое содержание
shorts = div.find('div', class_='block-text').text
print(shorts)
# block-text

Ему было 97 лет.


In [12]:
# Ссылку ищем как элемент словаря:
# link = div.a['href']
link = div.find('a', class_ = 'noshrink')['href']
# link = articles.a['href']

print(link)
# print(title, link)

/offline/1818637-umer-otec-nutella-himik-franchesko-rivella


In [24]:
# Время публикации:
time_pub = div.find('a', class_ = 'noshrink')
print(time_pub)

<a class="noshrink" data-router-link="" href="/offline/1818637-umer-otec-nutella-himik-franchesko-rivella"><!--[--><time datetime="2025-02-17T18:18:34.000Z" title="17.02.2025 в 21:18">вчера</time><!--]--></a>


In [33]:
# Время публикации:
# time_pub = div.find('a', class_ = 'noshrink').['time datetime']
# time_pub = div.find('div', class_ = 'content-header-details__inner').find('time')
time_dt = div.find('div', class_ = 'content-header-details__inner').find('time')['datetime']
time_t = div.find('div', class_ = 'content-header-details__inner').find('time')['title']
print(time_dt, time_t)
# print(title, link)

2025-02-17T18:18:34.000Z 17.02.2025 в 21:18


## ЭТО СКАЧАННЫЙ КОД !!!

In [19]:
# Достаём все заголовки статей со ссылками (используем цикл):
# Объект SOUP
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')
host = variable()[1][0]
articles = soup.find('div', class_='content-list')
divs = articles.find_all('div', class_='content--short')
count = 1
for art in divs:
    author = art.find('a', class_='author__name').text.strip()
    date = art.find('div', class_ = 'content-header-details__inner').find('time')['title']
    title = art.find('div', class_='content-title').text.strip() if art.find('div', class_='content-title') else 'Absent'
    shorts = art.find('div', class_='block-text').text.strip() if art.find('div', class_='block-text') else 'Absent'
    link = host + art.find('a', class_ = 'noshrink')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f'{count}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
    count +=1

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(VC)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	HTML-code "news_articles_vc" is OPENED from ../data/interim/html/news_articles_vc.html
Fun.3.0. "READ_HTML(VC)" COMPLETED SUCCESSFULLY!

1. Полина Лааксо - 18.02.2025 в 19:25
The New York Times «одобрило» использование ИИ в новостном и продуктовом отделах — Semafor
Издание задумалось об этом ещё в 2024 году.
https://vc.ru/ai/1820574-the-new-york-times-odobrilo-ispolzovanie-ii-v-novostnom-i-produktovom-otdelah-semafor
2025-02-18 20:00:03

2. Никита Якушкин - 18.02.2025 в 19:24
Самые упоминаемые нейросети в 2025 году
С помощью системы “Медиалогия” был проведен анализ того, как появление DeepSeek-R1 повлияло на обсуждение нейросетей в социальных сетях в 2025 году. Результаты исследования показывают, что текстовые и визуальные нейросети остаются в центре внимания пользователей, при этом ChatGPT сохраняет лидерство, а DeepSeek стрем

## ЭТО ИНТЕРНЕТ - ONLINE !!!

In [ ]:
from selenium import webdriver

In [12]:
# Рабочий вариант без скроллинга

headers = variable()[0]
host = variable()[1][0]
url = variable()[2][0]
articles = soup.find('div', class_='content-list')
divs = articles.find_all('div', class_='content--short')

r = requests.get(url=url, headers=headers)
src = r.text
soup = BeautifulSoup(src, 'lxml')
articles = soup.find('div', class_='content-list')
divs = articles.find_all('div', class_='content--short')

count = 1
page=1
print(page)
for art in divs:
    author = art.find('a', class_='author__name').text.strip()
    date = art.find('div', class_ = 'content-header-details__inner').find('time')['title']
    title = art.find('div', class_='content-title').text.strip() if art.find('div', class_='content-title') else 'Absent'
    shorts = art.find('div', class_='block-text').text.strip() if art.find('div', class_='block-text') else 'Absent'
    link = host + art.find('a', class_ = 'noshrink')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
    print(f'{count}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
    count +=1

1


AttributeError: 'NoneType' object has no attribute 'text'

In [52]:
# Selenium & скроллинг - пока не работает !!!
# from selenium import webdriver

headers = variable()[0]
host = variable()[1][0]
url = variable()[2][0]
articles = soup.find('div', class_='content-list')
divs = articles.find_all('div', class_='content--short')

r = requests.get(url=url, headers=headers)
src = r.text
soup = BeautifulSoup(src, 'lxml')
driver = webdriver.Chrome()

for item in range(3):
    print(item +1)
    for art in divs:
        author = art.find('a', class_='author__name').text
        date = art.find('div', class_ = 'content-header-details__inner').find('time')['title']
        title = art.find('div', class_='content-title content-title--low-indent').text
        shorts = art.find('div', class_='block-text').text
        link = host + art.find('a', class_ = 'noshrink')['href']
        date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        print(f'{author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')

    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    time.sleep(5) 
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

1
Евгения Евсеева - 18.02.2025 в 12:30
РСПП предложил ввести семейное налогообложение и индексировать пороги доходов для начисления НДФЛ 
Чтобы налог рассчитывался исходя из семейного положения, а порог для прогрессивного налога рос каждый год. 
https://vc.ru/money/1819449-rspp-predlozhil-vvesti-semeinoe-nalogooblozhenie-i-indeksirovat-porogi-dohodov-dlya-nachisleniya-ndfl
2025-02-18 13:40:40

Евгения Евсеева - 18.02.2025 в 11:23
«Магнит» проведёт коллаборацию с игрой Genshin Impact — с подарками в виде мерча и внутриигровых бонусов 
Она пройдёт с 12 марта по 22 апреля 2025 года.
https://vc.ru/marketing/1819325-magnit-provedet-kollaboraciyu-s-igroi-genshin-impact-s-podarkami-v-vide-mercha-i-vnutriigrovyh-bonusov
2025-02-18 13:40:41

Евгения Евсеева - 18.02.2025 в 11:00
Сервис для монетизации сообществ во «ВКонтакте» VK Donut добавил уровни подписок 
Настраивать их и выбирать, какой контент сделать доступным, будут сами авторы.
https://vc.ru/social/1819267-servis-dlya-monetizacii-soobsh

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=50694): Read timed out. (read timeout=120)

In [32]:
from selenium import webdriver

from time import sleep
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(cache_valid_range=10).install()))

driver = webdriver.Chrome() # или webdriver.Firefox()
url = 'https://vc.ru/new'


count = 1
page=1

while page < 3:
    print(page)
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(5)
    
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    articles = soup.find('div', class_='content-list')
    divs = articles.find_all('div', class_='content--short')
    
    for art in divs:
        author = art.find('a', class_='author__name').text.strip()
        date = art.find('div', class_ = 'content-header-details__inner').find('time')['title']
        title = art.find('div', class_='content-title').text.strip() if art.find('div', class_='content-title') else 'Absent'
        shorts = art.find('div', class_='block-text').text.strip() if art.find('div', class_='block-text') else 'Absent'
        link = host + art.find('a', class_ = 'noshrink')['href']
        date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        print(f'{count}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
        count +=1

    page +=1

1
1. Артем Колисниченко - 18.02.2025 в 20:27
Повышаем повторные продажи: 5 способов удержать клиента
Многие компании упускают возможность повторных продаж. Менеджеры забывают перезвонить, заявки теряются, а списки клиентов хранятся в хаотичном виде. В итоге теплый клиент уходит к конкурентам, а бизнес теряет доход. В статье расскажем, как упорядочить процессы, чтобы клиенты возвращались после первой покупки.
https://vc.ru/u/4179928-artem-kolisnichenko/1820753-povyshaem-povtornye-prodazhi-5-sposobov-uderzhat-klienta
2025-02-18 20:33:08

2. Вова Козырев - 18.02.2025 в 20:20
Как снизить стоимость привлечения соискателя в 3 раза: кейс клиента HR-tech компания.
Рассказываю, как помогаю компании по аутсорсингу персонала в 55 городах РФ, навести порядок в маркетинге и увеличить поток квал.лидов.
https://vc.ru/marketing/1820643-kak-snizit-stoimost-privlecheniya-soiskatelya-v-3-raza-keis-klienta-hr-tech-kompaniya
2025-02-18 20:33:09

3. Мышление предпринимателя - 18.02.2025 в 20:20
«Я так много

KeyboardInterrupt: 

In [31]:
count = 1
page=1

while page < 3:
    print(page)
    for i in range(1,13):
        print(count, end=', ')
        count +=1
    print()
    page+=1

1
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
2
13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 


# FORUM

In [ ]:
# Если нужно прокрутить к целевому элементу, можно воспользоваться методом .move_to_element() из ActionChains, пример:

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
target = driver.find_element_by_link_text('BROWSE TEMPLATES')
actions = ActionChains(driver)
actions.move_to_element(target)
actions.perform()

In [ ]:
# Для прокрутки в конец (начало) страницы через .send_keys():

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
driver.find_element_by_tag_name('body').send_keys(Keys.END)

# send_keys(Keys.HOME), если в начало
# так тоже сработает:
# send_keys(Keys.DOWN) send_keys(Keys.PAGE_DOWN)
# send_keys(Keys.UP) send_keys(Keys.PAGE_UP)

In [ ]:
# Для прокрутки на заданное количество пикселей метод окна .scrollBy():

scroll_vаlue = -200
scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
driver.execute_script(scroll_by)

In [ ]:
# Второй вариант цикл скроллинга

from selenium.webdriver.common.keys import Keys

while True:
    # прокручиваем страницу до конца
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    # проверяем, появились ли новые товары
    if driver.find_elements(By.CLASS_NAME, "tovar"):
        with open("C:/Users/SERGEY/PycharmProjects/ТEST/Test1", "w", encoding="utf=8") as file:
            file.write(driver.page_source)
    else:
        break